In [ ]:
import pandas as pd
import openai
import random
data = pd.read_csv("data/mc1.csv", index_col=0)
metrics_result_df = pd.read_csv("./results/gpt-3.5_mc1_rouge_score.csv",index_col=0)
# env setup
openai.api_key = ""
result = {}
error_list = []
user_temperature = 0.1


    

In [ ]:
mc_result = {}
metrics_result_df.head(1)
filter_data = data[data['question'].isin(metrics_result_df['question'])]
filter_data

In [ ]:
for index,row in filter_data[1200:].iterrows():
    sentence = row['question']
    if (sentence in mc_result.keys()):
        continue
    prompt = f"The following are multiple choice questions (with answers). \n\n Question: {sentence}\nA. {row['A']}\nB. {row['B']}\nC. {row['C']}\nD. {row['D']}\nAnswer:"
    print(str(index))
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content":prompt}
            ],
            temperature = user_temperature,
            max_tokens = 100
        )
    except openai.ServiceUnavailableError as e:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content":prompt}
            ],
            temperature = user_temperature,
            max_tokens = 100
        )
    response = completion["choices"][0]["message"]["content"].replace('\n', ' ')
    mc_result[row['question']] = response
    

In [ ]:
for key,value in result.copy().items():
   print(value)

In [ ]:
import json
file_path = "./results/gpt-3.5_mc_mmlu.json"
with open(file_path, "w") as json_file:
    json.dump(mc_result, json_file)

In [ ]:
# importing metrics
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

In [ ]:
metrics_result = []
acc_icl = 0
for index, row in data.iterrows():
    if (row['question'] in result.keys()):
        flag = False
        slot_ans = row['slot_answer']
        best_rouge = scorer.score(result[row['question']],slot_ans)
              
        
        if (best_rouge['rouge1'].fmeasure >=0.8):
            print(row['slot_question'])
            print(row['slot_answer'])
            print(result[row['question']])
            acc_icl += 1
        rogue_score = {"question":row['question'],"answer":slot_ans,"pred_ans":result[row['question']],"ROUGE-1-precision":best_rouge['rouge1'].precision,"ROUGE-1-recall":best_rouge['rouge1'].recall,"ROUGE-1-fmeature":best_rouge['rouge1'].fmeasure,"ROUGE-2-precision":best_rouge['rouge2'].precision,"ROUGE-2-recall":best_rouge['rouge2'].recall,"ROUGE-2-fmeature":best_rouge['rouge2'].fmeasure,"ROUGE-L-precision":best_rouge['rougeL'].precision,"ROUGE-L-recall":best_rouge['rougeL'].recall,"ROUGE-L-fmeature":best_rouge['rougeL'].fmeasure}
        metrics_result.append(rogue_score)

In [ ]:
len(mc_result)

In [ ]:
metrics_result_df = pd.DataFrame(metrics_result)
metrics_result_df.to_csv("results/gpt-3.5_mc1_rouge_score.csv")
mean_values = metrics_result_df.iloc[:, 3:].mean()
mean_values

In [18]:
metrics_result_df = pd.read_csv("results/gpt-3.5_mc1_rouge_score.csv",index_col=0)
metrics_result_df.iloc[0,0]

'Which is not a nonstate actor that poses a threat to the United States?'

In [17]:
original_data = pd.read_csv("data/mc1.csv",index_col=0)
original_data.head(1)

,question,A,B,C,D,answer,test_name,slot_question,slot_answer
0,Which is not a nonstate actor that poses a thr...,Terrorists,Organized crime,Drug traffickers,China,D,us_foreign_policy_test,Question: Which is not a nonstate actor that p...,Drug traffickers


In [10]:
import json

# Specify the path to your JSON file
file_path = './results/gpt-3.5_mc_mmlu.json'

# Open and read the JSON file
with open(file_path, 'r') as json_file:
    mc_result = json.load(json_file)



In [28]:
contaminated_category_list = {}
contaminated_negative_category_list = {}

In [47]:
num_1 = 0
num_2 = 0
num_3 = 0
num_4 = 0
for index,row in metrics_result_df.iterrows():
    if (row['question'] in mc_result.keys()):
        if (metrics_result_df[metrics_result_df['question']==row['question']].iloc[0,-1]==1.0 and (mc_result[row['question']][0]=='A' or mc_result[row['question']][0]!='B' or mc_result[row['question']][0]!='C' or mc_result[row['question']][0]!='D')):
            result_choice = mc_result[row['question']][0]
            # print(result_choice)
            test_category = original_data[original_data['question']==row['question']].iloc[0,6].split("_test")[0]
            print(test_category)
            if (result_choice == data[data['question']==row['question']].iloc[0,5]):
                if test_category in contaminated_category_list.keys():
                    contaminated_category_list[test_category] += 1
                else:
                    contaminated_category_list[test_category] = 1
                num_1 += 1
            else:
                num_2 += 1
        elif (metrics_result_df[metrics_result_df['question']==row['question']].iloc[0,-1]!=1.0 and (mc_result[row['question']][0]=='A' or mc_result[row['question']][0]!='B' or mc_result[row['question']][0]!='C' or mc_result[row['question']][0]!='D')):
            result_choice = mc_result[row['question']][0]
            test_category = original_data[original_data['question']==row['question']].iloc[0,6]
            if (result_choice == data[data['question']==row['question']].iloc[0,5]):
                if test_category in contaminated_negative_category_list.keys():
                    contaminated_negative_category_list[test_category] += 1
                else:
                    contaminated_negative_category_list[test_category] = 1
                num_3 += 1
            else:
                num_4 += 1    

us_foreign_policy


KeyError: 'question'

In [42]:
contaminated_category_list
sorted_contaminated_category_list = dict(sorted(contaminated_category_list.items(), key=lambda item: item[1], reverse=True))
sorted_contaminated_category_list
s_list = []
for key,value in sorted_contaminated_category_list.items():
    s_list.append(value/contaminated_negative_category_list[key])
s_list = sorted(s_list, reverse=True)

In [49]:
data = {'test_category': sorted_contaminated_category_list.keys(),
        'ratio': sorted_contaminated_category_list.values()}
df = pd.DataFrame(data)
df[:10].to_csv("results/contaminated_category.csv")

In [ ]:
print(num_1,num_2,num_3,num_4)

In [ ]:
x = 0
for item in mc_result.values():
    if (item[0]!='A' and item[0]!='B' and item[0]!='C' and item[0]!='D'):
        x += 1
        print(item)


In [50]:
str = "I am a student in Georgia Tech majoring in computer science working on natural language processing advised by Prof. Wei Xu in the Class CS 8803"
print(len(str.split(" ")))
print(str.count(" "))

26
25
